<a href="https://colab.research.google.com/github/9535432195/Sahana-/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install spacy transformers pdfplumber
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [15]:
import spacy
from transformers import pipeline
import pdfplumber

# Load SpaCy's English model and the BART summarization pipeline
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Function to extract key clauses based on keywords
def extract_key_clauses(legal_text):
    doc = nlp(legal_text)
    key_clauses = [
        sent.text for sent in doc.sents
        if "agreement" in sent.text.lower() or "breach" in sent.text.lower()
    ]
    return key_clauses

# Function to summarize legal text
def summarize_legal_text(legal_text):
    summary = summarizer(legal_text, max_length=150, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Main function to process legal text from a PDF or inline text
def process_legal_document(pdf_path=None, legal_text=None):
    if pdf_path:
        legal_text = extract_text_from_pdf(pdf_path)

    if not legal_text:
        raise ValueError("No legal text or PDF provided.")

    # Extract key clauses
    key_clauses = extract_key_clauses(legal_text)

    # Generate a summary
    summary = summarize_legal_text(legal_text)

    return {
        "key_clauses": key_clauses,
        "summary": summary
    }

# Example: Use uploaded PDF or inline text for testing
legal_text = """
This Agreement is made effective as of the date of signature, by and between Party A and Party B.
Both parties agree to the terms outlined, including but not limited to confidentiality, non-disclosure,
and intellectual property clauses. In case of breach, the party in violation will be subject to legal action.
This Agreement is governed by the laws of the state.
"""

# Process the document (choose one: uploaded PDF or inline text)
result = process_legal_document(legal_text=legal_text)  # Inline text example
# result = process_legal_document(pdf_path=pdf_path)  # If using uploaded PDF

# Display the results
print("Key Clauses:", result['key_clauses'])
print("Summary:", result['summary'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Your max_length is set to 150, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)


Key Clauses: ['This Agreement is made effective as of the date of signature, by and between Party A and Party B.\nBoth parties agree to the terms outlined, including but not limited to confidentiality, non-disclosure, \nand intellectual property clauses.', 'In case of breach, the party in violation will be subject to legal action.\n', 'This Agreement is governed by the laws of the state.\n']
Summary: Both parties agree to the terms outlined, including but not limited to confidentiality, non-disclosure,  and intellectual property clauses. In case of breach, the party in violation will be subject to legal action.
